In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer

In [ ]:
def carregando_csv():
    
    # Lendo o txt
    df = pd.read_csv(r"train.txt", sep=";")

    # Renomeando colunas
    df.columns = ["mensagem", "emocao"]

    # Transformando em csv
    df.to_csv("train.csv", index=False)

    print("Arquivo convertido e salvo como csv")
    
    return df

### Pré processamento dos Dados

In [ ]:
def processando_dados(df):

    # Retirando letras maiúsculas
    df["mensagem"] = df["mensagem"].str.lower()

    # Retirando palavras 'stopwords'
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))

    df["mensagem"] = df["mensagem"].apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

    # Lematiza as palavras
    nltk.download('wordnet')
    nltk.download('omw-1.4')

    lemmatizer = WordNetLemmatizer()
    df["mensagem"] = df["mensagem"].apply(lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split()))

    return df

## Modelos

### Bag-of-Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def execute_bow(df):

    X = df["mensagem"]
    y = df["emocao"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Usando a representação Bag of Words em X
    vectorizer = CountVectorizer()
    X_train_bow = vectorizer.fit_transform(X_train)
    X_test_bow = vectorizer.transform(X_test)

    # Criando e treinando o modelo
    model = MultinomialNB()
    model.fit(X_train_bow, y_train)

    # Fazendo previsões
    y_pred = model.predict(X_test_bow)

    return y_test, y_pred

### TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

def execute_tfidf(df):

    X = df['mensagem']
    y = df['emocao']

    # Convertendo mensagens para vetores TF-IDF
    vectorizer = TfidfVectorizer()
    X_tfidf = vectorizer.fit_transform(X)

    # Dividindo os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

    # Criando e treinando o modelo (aqui com RandomForest, mas pode ser outro classificador)
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Fazendo previsões
    y_pred = model.predict(X_test)

    return y_test, y_pred

### N-grams

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def execute_ngrams(df):


    X = df['mensagem']
    y = df['emocao']

    # Transformando em vetores N-grams (fazer testes depois para ver qual o melhor N, a acurácia tá baixa)
    vectorizer = CountVectorizer(ngram_range=(1, 2))

    # Vetorizando as mensagens
    X_ngrams = vectorizer.fit_transform(X)

    # Dividindo os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X_ngrams, y, test_size=0.2, random_state=42)

    # Criando e treinando o modelo
    model = MultinomialNB()
    model.fit(X_train, y_train)

    # Fazendo previsões
    y_pred = model.predict(X_test)


    return y_test, y_pred

### Word embeddings

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def execute_word_embeddings(df):

    X = df['mensagem']
    y = pd.get_dummies(df['emocao']).values  # Convertendo classes para one-hot encoding

    # Tokenização
    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X)
    X_sequences = tokenizer.texts_to_sequences(X)
    X_padded = pad_sequences(X_sequences, maxlen=100)

    # Separação de dados
    X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

    # Modelo com embeddings
    model = Sequential([
        Embedding(input_dim=5000, output_dim=100, input_length=100),
        LSTM(64),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Treinamento
    model.fit(X_train, y_train, epochs=5, validation_split=0.2)

    # Predições
    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)


    return y_test_labels, y_pred_labels

### Subword embeddings

### BERT

## Main

In [ ]:
def main():

    df = carregando_csv()
    df = processando_dados(df)

    while True:
        
        print('Selecione uma das técnicas para prosseguir:')
        print('1 - Bag-of-Words')
        print('2 - TF-IDF')
        print('3 - N-grams')
        print('4 - Word embeddings')
        print('5 - Subword embeddings')
        print('6 - BERT')
        print('7 - Ver estatísticas')
        print('8 - Sair')

        opcao = input()
    
        if opcao == '1':
            y_test, y_pred = execute_bow(df)

            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

        elif opcao == '2':

            y_test, y_pred = execute_tfidf(df)

            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

        elif opcao == '3':
            y_test, y_pred = execute_ngrams(df)

            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

        elif opcao == '4':
            y_test, y_pred = execute_word_embeddings(df)
            
            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

        elif opcao == '5':
            y_test, y_pred = execute_subword_embeddings(df)

            print("Acurácia:", accuracy_score(y_test, y_pred))
            print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))

        elif opcao == '6':
            print('Em construção')

        elif opcao == '7':
            print('Em construção')

        elif opcao == '8':
            break
        
        else:
            print('Opção inválida :O tente de novo!!')
        

if __name__ == "__main__":
    main()
    